<a href="https://colab.research.google.com/github/nitsundon/AIML/blob/main/XGboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [12]:
url="https://raw.githubusercontent.com/nitsundon/AIML/c7bd2bfabc3a39aa43a77736fd15368ce8ddf1cb/amr.csv"
df = pd.read_csv(url)


In [13]:
df.columns

Index(['datetime', 'index', 'MSEDCL', 'TATA POWER', 'AEML', 'BEST', 'RAILWAY',
       'MBPPPL SEZ', 'GEPL SEZ', 'NIDAR SEZ', 'KRC', 'EON KHARADI (Ph-1)',
       'sum', 'block_number', 'TestD1', 'LBSCML', 'EON KHARADI (Ph-2)', 'MADC',
       'JNPT', 'AEML SEEPZ Ltd(ASL)', 'HADAPSAR SEZ', ' MANJARI SEZ'],
      dtype='object')

In [17]:
df=df[['datetime','sum']]


In [ ]:
df["Hour"] = df["Timestamp"].dt.hour
df["DayOfWeek"] = df["Timestamp"].dt.dayofweek
df["Month"] = df["Timestamp"].dt.month
df["Quarter"] = df["Timestamp"].dt.quarter